In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
### 卷积计算
#### 卷积为正方形
# n * n input  
# f * f filter  
# padding p stride s  
# 输出大小O为：  
#         O = (n-f+2p)/s  + 1


In [2]:
class ChannelAttention(nn.Module):
    def __init__(self,in_planes):
        super(ChannelAttention,self).__init__()

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc1 = nn.Conv2d(in_planes,in_channels // 16,kernel_size=1,bias=False)
        self.relu = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // 16,in_planes,kernel_size=1,biase=False)
        self.sigmod = nn.Sigmoid()

    def forward(self,x):
        
        avg_out = self.avg_pool(x)
        avg_out = self.fc1(avg_out)
        avg_out = self.relu(avg_out)
        avg_out = self.fc2(avg_out)
        
        max_out = self.max_pool(x)
        max_out = self.fc1(max_out)
        max_out = self.relu(max_out)
        max_out = self.fc2(max_out)

        out = avg_out + max_out
        out = self.sigmod(out)

        return out

In [3]:
class SpatialAttention(nn.Module):
    def __init__(self,in_planes):
        super(SpatialAttention,self).__init__()

        self.conv1 = nn.Conv2d(2,1,kernel_size=7,paddin=3,biase=False)
        self.sigmod = nn.Sigmoid()

    def forward(self,x):
        
        avg_out = torch.mean(x,dim=1,keepdim=True)
        
        max_out = torch.max(x,dim=1,keepdim=True)

        out = torch.cat([avg_out,max_out],dim=1)
        out = self.conv1(out)
        out = self.sigmod(out)

        return out

In [4]:
#18layer 34layer 结构
class BasicBlock(nn.Module):
    expansion = 1#用于标注是否需要转换卷积核个数
    def __init__(self,in_channel,out_channel,stride =1,downsample=None):
        #downsample 下采样 在高层网络中的 进行降维（宽高，非通道）的那个操作 56，56，64--*stride：2---1，1，128*----28，28，128
        super(BasicBlock,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel,out_channels=out_channel,
                                kernel_size=3,stride=stride,padding=1,bias=False)
        #若 stride=1 paddi=1则 宽高不会改变 output= (input -3 +2*1)/1 + 1 = input
        #若 stride=2 padding=1则： output = (input -3 + 2*1) /2 +1
        # = input/2 +0.5
        # = input/2(向下取整)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=out_channel,out_channels=out_channel,
                                kernel_size=3,stride=1,padding=1,bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.downsample = downsample

        self.channel = ChannelAttention(self.expansion*out_channel)
        self.spatial = SpatialAttention()
    
    def forward(self,t):
        identity = t
        if self.downsample is not None:
            identity = self.downsample(t)
        t = self.conv1(t)
        t = self.bn1(t)
        t = self.relu(t)

        t = self.conv2(t)
        t = self.bn2(t)
        
        t += identity
        t = self.relu(t)

        return t

In [5]:
# 参数inplace：是否选择进行覆盖运算，默认值为False。

# 若为True则进行覆盖运算，例如：x = x +1,即对原值进行操作，又将结果直接赋值给原值。

# 若为False则不进行直接覆盖运算，例如：y = x +1,x = y，即找一个中间按变量y作为传递

# 具体到上述代码，inplace=True时,就对self.conv计算得到的结果直接做修改，这样做的好处就是可以节省内存，还可以节省反复申请和释放内存的时间，且最后计算结果与inplace=False相同，只要不出现错误就可以使用。

#50layer 101layer 152layer 结构
class Bottlenect(nn.Module):
    expansion = 4#用于标注是否需要转换卷积核个数
    def __init__(self,in_channel,out_channel,stride =1,downsample=None):
        #downsample 下采样 在高层网络中的 进行降维（宽高，非通道）的那个操作 56，56，64--*stride：2---1，1，128*----28，28，128
        super(Bottlenect,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel,out_channels=out_channel,
                                kernel_size=1,stride=1,padding=1,bias=False)
        #若 stride=1 paddi=1则 宽高不会改变 output= (input -3 +2*1)/1 + 1 = input
        #若 stride=2 padding=1则： output = (input -3 + 2*1) /2 +1
        # = input/2 +0.5
        # = input/2(向下取整)
        self.bn1 = nn.BatchNorm2d(out_channel)
        
        #----------------------------------------------
        self.conv2 = nn.Conv2d(in_channels=out_channel,out_channels=out_channel,
                                kernel_size=3,stride=1,padding=1,bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)

        #------------------------------------------------
        self.conv3 = nn.Conv2d(in_channels=out_channel,out_channels=out_channel*self.expansion,
                                kernel_size=1,stride=1,bias=False)
        self.bn3 = nn.BatchNorm2d(out_channel*self.expansion)

        self.relu = nn.ReLU(implace=True)
        self.downsample = downsample
    
    def forward(self,t):
        identity = t
        if self.downsample is not None:
            identity = self.downsample(t)
        t = self.conv1(t)
        t = self.bn1(t)
        t = self.relu(t)

        t = self.conv2(t)
        t = self.bn2(t)
        t = self.relu(t)

        t = self.conv3(t)
        t = self.bn3(t)
        
        t += identity
        t = self.relu(t)

        return t


In [ ]:
class CBAM_ResNet(nn.Module):
    def __init__(self,block,blocks_num,num_classes =1000,include_top=True):
        super(CBAM_ResNet,self).__init__()
        self.include_top = include_top#是否包含全连接层 用于后期搭建更为复杂的网络
        self.in_channel = 64

        self.conv1 = nn.Conv2d(3,self.in_channel,kernel_size=7,stride=2,
                                padding=3,bias=False)#nn.conv2d中第一个参数就是输入channel 第二个参数为输出的channel
        #输入channel3 表示 rgb彩色图像
        #O = (n-f+2p)/s  + 1      in-224 out-112  (224-7+2p)2 + 1 =112
        #p = 2 或 3  带回去算 向下取整 所以用 3 
        self.bn1 = nn.BatchNorm2d(self.in_channel)
        self.relu = nn.ReLU(inplace = True)
        #pool后宽高为56*56
        self.maxpool = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

        #block就是 basicblock（不需要转换为度 18 layer &34 layer 或者 bottlenect（需要转换维度）
        self.layer1 = self._make_layer(block,64, blocks_num[0])
        self.layer2 = self._make_layer(block,128, blocks_num[1],stride = 2)
        self.layer3 = self._make_layer(block,256, blocks_num[2],stride = 2)
        self.layer4 = self._make_layer(block,512, blocks_num[3],stride =2)

        if self.include_top:
            self.avgpool = nn.AdaptiveAvgPool2d((1,1))
            self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m,nn.Conv2d):
                nn.init.kaiming_normal_(m.weight,mode= 'fan_out', nonlinearity='relu')


    def _make_layer(self,block, channel, block_num,stride=1):
        downsample = None

        #18 和 34层会跳过下 if语句   self.in_channel =64 若是需要改变维度启用bottleneck则bottle中的expansion为4 
        #layer2 中没有传入参数 stride stirde默认为1
        if stride!=1 or self.in_channel != channel*block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channel,channel * block.expansion, kernel_size=1,
                stride = stride, bias= False),
                nn.BatchNorm2d(channel * block.expansion)
            )
        
        layers = []
        layers.append(block(self.in_channel,channel,downsample=downsample,stride=stride))
        self.in_channel = channel*block.expansion# 50 101 152的每个block的第三层的channel数量为1 2 层的4倍

        for _ in range(1,block_num):
            layers.append(block(self.in_channel,channel))
        
        return nn.Sequential(*layers)

    def forward(self,t):
        t = self.conv1(t)
        t = self.bn1(t)
        t = self.relu(t)
        t = self.maxpool(t)

        t = self.layer1(t)
        t = self.layer2(t)
        t = self.layer3(t)
        t = self.layer4(t)

        if self.include_top:
            t = self.avgpool(t)
            t = torch.flatten(t,1)
            t = self.fc(t)
        
        return t


In [ ]:
def resnet18 (num_classes=1000,include_top = True):
        return ResNet(BasicBlock,[2,2,2,2],num_classes=num_classes,include_top=include_top)


def resnet34 (num_classes=1000,include_top = True):
        return ResNet(BasicBlock,[3,4,6,3],num_classes=num_classes,include_top=include_top)

def resnet50 (num_classes=1000,include_top = True):
        return ResNet(Bottlenect,[3,4,6,3],num_classes=num_classes,include_top=include_top)


def resnet101 (num_classes=1000,include_top = True):
        return ResNet(Bottlenect,[3,4,23,3],num_classes=num_classes,include_top=include_top)


#import torchvision.models.resnet